In [2]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from wand.image import Image as WImage
from scipy.stats import ks_2samp, ttest_ind, shapiro

In [3]:
def get_dwell_median(path):
    """
    gets the median dwell time for individual reads in a dict from AWS emr output
    """
    df = pd.read_csv(path)
    df.columns = ["read_name", "median_dwell"]
    return df.set_index("read_name")["median_dwell"].to_dict()

def get_df(prefix,contig, positions,median_profile_path):
    """
    put positions from the distributed file system
    into one dataframe
    """
    df = pd.DataFrame()
    for pos in positions:
        
        path = prefix + 'contig=' + contig + '/position=' + str(pos -1)
        csv = glob.glob(path + "/*.csv")
        columns = ['read_name','kmer','model_mean','model_stdv','dwell','event_mean']
        new_df = pd.read_csv(csv[0], header=None)
        new_df.columns = columns
        new_df['position'] = pos
        df = df.append(new_df)
        
    median_dwells = get_dwell_median(median_profile_path)
    df["med_dwell"] = df["read_name"].map(median_dwells)
    df["normalized_dwell"] = df["dwell"]/df["med_dwell"]
    df.drop(columns=["med_dwell","dwell"],inplace=True)
#     df['dwell'] = df['dwell'] * 1000 # get dwell in ms
    return df


def get_stat(native, ivt, contig, position, output_file):
    """
    takes the native and ivt dataframes and outputs some stats (KS, ttest, )
    """
    num_native = len(native)
    num_ivt = len(ivt)
    kmer = native.kmer.iloc[0]
    kmer = kmer.replace('T','U')
    signal_ks, signal_ksP = ks_2samp(np.ravel(native["event_mean"]),np.ravel(ivt["event_mean"]))
    dwell_ks, dwell_ksP = ks_2samp(np.ravel(native["normalized_dwell"]),np.ravel(ivt["normalized_dwell"]))
    dwell_tt, dwell_ttP = ttest_ind(np.ravel(native["normalized_dwell"]),np.ravel(ivt["normalized_dwell"]),equal_var=False)
    signal_tt, signal_ttP = ttest_ind(np.ravel(native["event_mean"]),np.ravel(ivt["event_mean"]),equal_var=False)
    native_shap,p = shapiro(np.ravel(native["event_mean"]))
    ivt_shap,p = shapiro(np.ravel(ivt["event_mean"]))

    out = [contig, position , kmer, num_ivt, num_native, signal_ks, signal_ksP, signal_tt,signal_ttP,ivt_shap, native_shap,
           ivt["event_mean"].mean(), native["event_mean"].mean(), ivt["event_mean"].median(), native["event_mean"].median(),
           ivt["event_mean"].std(), native["event_mean"].std(), dwell_ks, dwell_ksP, dwell_tt, dwell_ttP, ivt["normalized_dwell"].mean(), 
           native["normalized_dwell"].mean(),ivt["normalized_dwell"].median(), native["normalized_dwell"].median(), ivt["normalized_dwell"].std(), native["normalized_dwell"].std()]

    out = "\t".join([str(i) for i in out])
    outF = open(output_file, "a")
#     print("\t".join(["contig", "position", "kmer", "num_reads_ivt", "num_reads_native", "ks_signal", "ks_signal_pval",
#                     "ttest_signal", "ttest_signal_pval", "signal_shapiro_ivt", "signal_shapiro_ivt", "signal_mean_ivt", "signal_mean_native",
#                      "signal_median_ivt","signal_median_native", "signal_std_ivt", "signal_std_native", "ks_dwell","ks_dwell_pval", 
#                     "ttest_dwell", "ttest_dwell_pval", "dwell_mean_ivt", "dwell_mean_native", "dwell_median_ivt",
#                     "dwell_median_native", "dwell_std_ivt", "dwell_std_native" ]), end="\n", file=outF)
    print(out, end="\n", file=outF)
    outF.close()

In [90]:
native = get_df("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_native/","ecoli23S",[10])
ivt = get_df("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_ivt/","ecoli23S",[10])

In [86]:
# get_stat(native, ivt,"ecoli23S" , 8, "stats/test1.tsv")
ks_2samp(native["event_mean"],ivt["event_mean"],mode="asymp")

Ks_2sampResult(statistic=0.08486387920352378, pvalue=5.299003133011954e-08)

In [63]:
def parser(path_native, path_ivt, path_median_dwell_native, path_median_dwell_ivt, contig, end_coor, save_path):
    
    outF = open(save_path, "a")
    print("\t".join(["contig", "position", "kmer", "num_reads_ivt", "num_reads_native", "ks_signal", "ks_signal_pval",
                    "ttest_signal", "ttest_signal_pval", "signal_shapiro_ivt", "signal_shapiro_ivt", "signal_mean_ivt", "signal_mean_native",
                     "signal_median_ivt","signal_median_native", "signal_std_ivt", "signal_std_native", "ks_dwell","ks_dwell_pval", 
                    "ttest_dwell", "ttest_dwell_pval", "dwell_mean_ivt", "dwell_mean_native", "dwell_median_ivt",
                    "dwell_median_native", "dwell_std_ivt", "dwell_std_native" ]), end="\n", file=outF)
    outF.close()
    for pos in range(1,end_coor+1):
        
        native = get_df(path_native, contig, [pos], path_median_dwell_native)
        ivt = get_df(path_ivt, contig, [pos], path_median_dwell_ivt)
        get_stat(native, ivt, contig, pos, save_path)
        

In [ ]:
parser("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_native/","/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_ivt/",
       "1623_native_median_dwell_byRead.csv", "1623_ivt_median_dwell_byRead.csv",
      "ecoli23S", 2900, "stats/ecoli23S_normalized_dwell.tsv")
print("Finished!")

In [ ]:
parser("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_native/","/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_ivt/",
       "1623_native_median_dwell_byRead.csv", "1623_ivt_median_dwell_byRead.csv",
      "ecoli16S", 1538, "stats/ecoli16S_normalized_dwell.tsv")
print("Finished!")

In [ ]:
parser("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1825_native/","/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1825_ivt/",
       "1825_native_median_dwell_byRead.csv", "1825_ivt_median_dwell_byRead.csv",
      "sc25S", 3392, "stats/sc25S_normalized_dwell.tsv")
print("Finished!")

In [ ]:
parser("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1825_native/","/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1825_ivt/",
       "1825_native_median_dwell_byRead.csv", "1825_ivt_median_dwell_byRead.csv",
      "sc18S", 1796, "stats/sc18S_normalized_dwell.tsv")
print("Finished!")

/home/roham/.local/lib/python3.6/site-packages/scipy/stats/morestats.py:1660: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [96]:
def plot_stat(df, title, save_path, col):
    plt.figure(figsize=(30, 6))
    sns.lineplot(x="position",y=col,data=df)
    sns.despine()
    # plt.title("Control_Structure")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(save_path + title + '.pdf', format='pdf', dpi=1200,bbox_inches='tight')

# operon specific signal analysis

In [4]:
def sammap(file):
    df = pd.read_csv(file , sep=" " ,comment='@', header=None)
    df.columns = ["read_name", "contig", "mapQ"]
    df = df[df["mapQ"] != 0 ]
    s = df.groupby(['read_name'], sort=False)['mapQ'].max()
    idx = df.groupby(['read_name'])['mapQ'].transform(max) == df['mapQ']
    s = df[idx]
    s = s.drop_duplicates("read_name", keep=False)
    return s

def operon_reads(df, sammap_output, contig):
    reads = sammap_output[sammap_output["contig"] == contig].read_name.to_list()
    df = df[df["read_name"].isin(reads)]
    
    return df

def parser_operons(path_native, path_ivt, path_median_dwell_native, path_median_dwell_ivt,
           sammap_native_file, sammap_ivt_file, contig, end_coor, save_path):
    smap_native= sammap(sammap_native_file)
    smap_ivt = sammap(sammap_ivt_file)
#     outF = open(save_path + "_" + op + ".tsv", "a")
#     print("\t".join(["contig", "position", "kmer", "num_reads_ivt", "num_reads_native", "ks_signal", "ks_signal_pval",
#                     "ttest_signal", "ttest_signal_pval", "signal_shapiro_ivt", "signal_shapiro_ivt", "signal_mean_ivt", "signal_mean_native",
#                      "signal_median_ivt","signal_median_native", "signal_std_ivt", "signal_std_native", "ks_dwell","ks_dwell_pval", 
#                     "ttest_dwell", "ttest_dwell_pval", "dwell_mean_ivt", "dwell_mean_native", "dwell_median_ivt",
#                     "dwell_median_native", "dwell_std_ivt", "dwell_std_native" ]), end="\n", file=outF)
#     outF.close()
    for op in ['rrnA_23s', 'rrnBG_23s', 'rrnC_23s', 'rrnD_23s', 'rrnE_23s', 'rrnH_23s']:
        outF = open(save_path + "_" + op + ".tsv", "a")
        print("\t".join(["contig", "position", "kmer", "num_reads_ivt", "num_reads_native", "ks_signal", "ks_signal_pval",
                        "ttest_signal", "ttest_signal_pval", "signal_shapiro_ivt", "signal_shapiro_ivt", "signal_mean_ivt", "signal_mean_native",
                         "signal_median_ivt","signal_median_native", "signal_std_ivt", "signal_std_native", "ks_dwell","ks_dwell_pval", 
                        "ttest_dwell", "ttest_dwell_pval", "dwell_mean_ivt", "dwell_mean_native", "dwell_median_ivt",
                        "dwell_median_native", "dwell_std_ivt", "dwell_std_native" ]), end="\n", file=outF)
        outF.close()
    for pos in range(50,end_coor+1):
        for op in ['rrnA_23s', 'rrnBG_23s', 'rrnC_23s', 'rrnD_23s', 'rrnE_23s', 'rrnH_23s']:
            
            native = get_df(path_native, contig, [pos], path_median_dwell_native)
            native = operon_reads(native, smap_native, op)
            ivt = get_df(path_ivt, contig, [pos], path_median_dwell_ivt)
            ivt = operon_reads(ivt, smap_ivt, op)
            get_stat(native, ivt, contig, pos, save_path + "_" + op + ".tsv")
        

In [ ]:
parser_operons("/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_native/","/kyber/Data/Nanopore/projects/nano2om/190617_nano2om/aws_emr/1623_ivt/",
       "1623_native_median_dwell_byRead.csv", "1623_ivt_median_dwell_byRead.csv",
      "/kyber/Data/Nanopore/projects/nano2om/minimap2-23s/23s-native_sammap.txt",
       "/kyber/Data/Nanopore/projects/nano2om/minimap2-23s/23s-ivt_sammap.txt",
       "ecoli23S", 2900, "stats/ecoli23S_normalized_dwell")
print("Finished!")

/home/roham/.local/lib/python3.6/site-packages/scipy/stats/morestats.py:1660: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [61]:
k = sammap("/kyber/Data/Nanopore/projects/nano2om/minimap2-23s/23s-native_sammap.txt"))